# Advent of Code 2022

## Day 1

In [2]:
use std::fs;

#[derive(Debug, Eq, Ord, PartialEq, PartialOrd)]
struct Elf {
    calories: u32,
}

impl Elf {
    fn new(snacks_str: &str) -> Self {
        let snacks = snacks_str.lines().filter_map(|s| s.parse::<u32>().ok()).collect::<Vec<_>>();
        Self {
            calories: snacks.iter().sum::<u32>(),
        }
    }
}
    
fn elves_from_file(path: &str) -> Vec<Elf> {
    let file_str = fs::read_to_string(path).expect("failed to read");
    let elves_strs = file_str.split("\n\n");
    elves_strs.map(|e| Elf::new(e)).collect::<Vec<_>>()
}

In [3]:
// Part 1
elves_from_file("day1_input.txt").iter().max().unwrap().calories

69289

In [4]:
// Part 2
let mut elves = elves_from_file("day1_input.txt");
elves.sort();
elves.reverse();
elves[..3].iter().map(|e| e.calories).sum::<u32>()

205615

## Day 2

In [5]:
use std::fs;

enum RoundResult {
    Loss,
    Draw,
    Win,
}

impl RoundResult {
    fn value(&self) -> u32 {
        match *self {
            RoundResult::Loss => 0,
            RoundResult::Draw => 3,
            RoundResult::Win => 6,
        }
    }
}

enum Draw {
    Rock,
    Paper,
    Scissors,
}

impl Draw {
    fn new(draw: &str) -> Self {
        match draw {
            "A" => Draw::Rock,
            "B" => Draw::Paper,
            "C" => Draw::Scissors,
            "X" => Draw::Rock,
            "Y" => Draw::Paper,
            "Z" => Draw::Scissors,
            _ => panic!("invalid draw"),
        }
    }
    
    fn value(&self) -> u32 {
        match *self {
            Draw::Rock => 1,
            Draw::Paper => 2,
            Draw::Scissors => 3,
        }
    }
}

struct Round {
    opponent: Draw,
    my: Draw,
}

impl Round {
    fn new(round_str: &str) -> Self {
        let mut player_str = round_str.split(" ");
        let opponent_str = player_str.next().expect("Failed to get opponent from string");
        let my_str = player_str.next().expect("Failed to get myself from string");
        
        Self {
            opponent: Draw::new(opponent_str),
            my: Draw::new(my_str),
        }
    }
    
    fn determine_result(&self) -> RoundResult {
        match self.my {
            Draw::Rock => match self.opponent {
                Draw::Rock => RoundResult::Draw,
                Draw::Paper => RoundResult::Loss,
                Draw::Scissors => RoundResult::Win,
            }
            Draw::Paper => match self.opponent {
                Draw::Rock => RoundResult::Win,
                Draw::Paper => RoundResult::Draw,
                Draw::Scissors => RoundResult::Loss,
            }
            Draw::Scissors => match self.opponent {
                Draw::Rock => RoundResult::Loss,
                Draw::Paper => RoundResult::Win,
                Draw::Scissors => RoundResult::Draw,
            }
        }
    }
    
    fn follow_elf_guide(&self) -> u32 {
        let result = match self.my {
                Draw::Rock => RoundResult::Loss,     //X
                Draw::Paper => RoundResult::Draw,    //Y
                Draw::Scissors => RoundResult::Win,  //Z
        };
        let draw = match self.opponent {
            Draw::Rock => match result {
                RoundResult::Loss => Draw::Scissors,
                RoundResult::Draw => Draw::Rock,
                RoundResult::Win => Draw::Paper,
            },
            Draw::Paper => match result {
                RoundResult::Loss => Draw::Rock,
                RoundResult::Draw => Draw::Paper,
                RoundResult::Win => Draw::Scissors,
            },
            Draw::Scissors => match result {
                RoundResult::Loss => Draw::Paper,
                RoundResult::Draw => Draw::Scissors,
                RoundResult::Win => Draw::Rock,
            },
        };
        result.value() + draw.value()
    }
    
    fn points(&self) -> u32 {
        self.my.value() + self.determine_result().value()
    }
}

fn rounds_from_file(path: &str) -> Vec<Round> {
    let file_str = fs::read_to_string(path).expect("failed to read");
    let rounds_str = file_str.lines();
    rounds_str.map(|r| Round::new(r)).collect::<Vec<_>>()
}

In [6]:
// Part 1
rounds_from_file("day2_input.txt").iter().map(|r| r.points()).sum::<u32>()

10404

In [7]:
// Part 2
rounds_from_file("day2_input.txt").iter().map(|r| r.follow_elf_guide()).sum::<u32>()

10334

# Day 3

In [8]:
use std::fs;
use std::collections::HashSet;

fn calculate_priority(item: &char) -> u32 {
    match item.is_uppercase() {
        true => *item as u32 - 'A' as u32 + 27,
        false => *item as u32 - 'a' as u32 + 1,
    }
}

type SetType = HashSet<char>;

#[derive(Debug, Clone)]
struct Rucksack {
    all_items: SetType,
    compartment_1: SetType,
    compartment_2: SetType,
}

impl Rucksack {
    fn new(rucksack_str: &str) -> Self {
        let compartments_str = rucksack_str.split_at(rucksack_str.len() / 2);
        Self {
            all_items: HashSet::from_iter(rucksack_str.chars()),
            compartment_1: HashSet::from_iter(compartments_str.0.chars()),
            compartment_2: HashSet::from_iter(compartments_str.1.chars()),
        }
    }
    
    fn find_common_item(&self) -> &char {
        self.compartment_1.intersection(&self.compartment_2).next().expect("couldn't find common item")
    }

    fn intersect(&self, other: &Self) -> Self {
        Rucksack::new(&String::from_iter(self.all_items.intersection(&other.all_items))[..])
    }
    
    fn get_item(&self) -> Option<char> {
        if self.all_items.len() == 1 {
            return Some(*self.all_items.iter().next().unwrap())
        }
        None
    }
}

fn rucksacks_from_file(path: &str) -> Vec<Rucksack> {
    let file_str = fs::read_to_string(path).expect("failed to read rucksacks from file");
    let rucksack_str = file_str.lines();
    rucksack_str.map(|r| Rucksack::new(r)).collect::<Vec<_>>()
}

In [12]:
// Part 1
rucksacks_from_file("day3_input.txt").iter().map(|r| calculate_priority(r.find_common_item())).sum::<u32>()

7674

In [14]:
// Part 2
:dep itertools = "0.10.5"
use itertools::Itertools;
rucksacks_from_file("day3_input.txt").into_iter().chunks(3).into_iter().map(|g| calculate_priority(&g.into_iter().reduce(|a, b| a.intersect(&b)).map(|b| b.get_item().unwrap()).unwrap())).sum::<u32>()

2805